In [29]:
import pandas as pd
from tqdm import tqdm
import time
import json
import requests


df = pd.read_csv('./data/geographies_to_process.csv') 

tqdm.pandas()

df['type'] = ''
df['name'] = ''
df['lat'] = ''
df['lon'] = ''
df['raw'] = ''

regions = ['West', 'Plains', 'Northeast', 'South Central', 'Southeast', 'Total U.S.']
states = ['South Carolina', 'New York', 'California', 'West Tex/New Mexico']

def geocode(address_or_zipcode):
    GM_API_KEY = 'UPDATE WITH YOUR KEY HERE'
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={GM_API_KEY}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
    except:
        pass
    return results


for index, location in df.iterrows():
    if location['geography'] in regions:
        df.loc[index, 'location_type'] = 'region'
    elif location['geography'] in states:
        df.loc[index, 'location_type'] = 'state'
    else:
        df.loc[index, 'location_type'] = 'city'
        

    
    try:
        print("Geocoding for: ", location['location_name'])
        
        location = geocode(location['location_name'])
        #print(json.dumps(location, indent = 4, sort_keys=True)) # pretty print
        print(location) # line print
        
        df.loc[index, 'geography'] = location['formatted_address']
        df.loc[index, 'lat'] = location['geometry']['location']['lat']
        df.loc[index, 'lon'] = location['geometry']['location']['lng']
        df.loc[index, 'raw'] = json.dumps(location)
        
    except:
        print("Couldn't geocode: ", location['location_name'])
        
    time.sleep(1)

df.to_csv('./data/geographies_processed_new.csv', index=False)


Geocoding for:  Plains
Couldn't geocode:  Plains
Geocoding for:  South East
Couldn't geocode:  South East
Geocoding for:  West
Couldn't geocode:  West
Geocoding for:  Mid South
Couldn't geocode:  Mid South
Geocoding for:  Hartford
{'address_components': [{'long_name': 'Hartford', 'short_name': 'Hartford', 'types': ['locality', 'political']}, {'long_name': 'Hartford County', 'short_name': 'Hartford County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Connecticut', 'short_name': 'CT', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Hartford, CT, USA', 'geometry': {'bounds': {'northeast': {'lat': 41.807575, 'lng': -72.6425359}, 'southwest': {'lat': 41.7237999, 'lng': -72.717638}}, 'location': {'lat': 41.7658043, 'lng': -72.6733723}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 41.807575, 'lng': -72.6425359}, 'southwest': {'la